In [26]:
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.python.eager.context import graph_mode

In [27]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


Tính hog

In [28]:
# xây dựng hàm tính hog
def f_hog(x):
  f_hog_lis = []
  for i in range(len(x)):
      f = hog(x[i], orientations=9, pixels_per_cell=(4,4),cells_per_block=(2,2), visualize=False)
      f_hog_lis.append(f)
  return f_hog_lis

In [29]:
f_hog_train = f_hog(x_train)

In [30]:
f_hog_test = f_hog(x_test)

PCA data

In [31]:
e = 1e-9
imag_mean = np.mean(f_hog_train,axis = 0, keepdims=True)
imag_std = np.std(f_hog_train,axis = 0, keepdims=True)
f_hog_train -= imag_mean;   f_hog_train /= (imag_std + e)   
f_hog_test -= imag_mean;    f_hog_test /= (imag_std + e)

In [32]:
from sklearn import decomposition
n_components = 128
pca = decomposition.PCA(n_components=n_components)

In [33]:
# fit data
pca.fit(f_hog_train)
Z_train = pca.transform(f_hog_train)
Z_test = pca.transform(f_hog_test)

In [34]:
print(Z_train.shape)
Z_test.shape

(60000, 128)


(10000, 128)

training model

In [39]:
# xây dụng model 
model = tf.keras.models.Sequential()
#tạo ra 2 lớp của model với 128 điểm  
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
#tạo lớp cuối vói 10 điểm 
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [40]:
# compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(Z_train, y_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1442 - accuracy: 0.9561
Epoch 2/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0598 - accuracy: 0.9814
Epoch 3/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0424 - accuracy: 0.9865


kết quả accuracy

In [42]:
#loss function and accuracy function
val_loss, val_acc = model.evaluate(Z_test, y_test)
print(val_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.0713 - accuracy: 0.9787
0.9786999821662903
